# ATAP Notebook for formatting and merging reference location data

This notebook is the steps taken to merge and reformat the reference data about signiifcant locations, like current capital cities and coutries, and any similar historical location data.This notebook helps you access the Geolocation tools in a Python development environment.

## Requirements <a class="anchor" id="section-requirements"></a>

<div class="alert alert-block alert-info">
This notebook uses various Python libraries. Most will come with your Python installation, but the following are crucial:
<ul>
    <li> pandas</li> 
    <li> json</li> 
    <li> nltk</li> 
    <li> geopandas</li> 
    <li> shapely  </li> 
</ul>
</div>

In [1]:
# TODO: UPDATE
# Many of these are probably not needed.

import os
from pickle import NONE
import nltk
import csv
import time
import urllib
import requests
import json
import math

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Geopandas is used to work with spatial data
# If you have issues installing it on a MAcOS, 
# see https://stackoverflow.com/questions/71137617/error-installing-geopandas-in-python-on-mac-m1
import geopandas as gpd
from geopandas import GeoDataFrame

# NLTK is used to work with textual data 
#from nltk.tag import StanfordNERTagger
#from nltk.tokenize import word_tokenize

# spaCy is used for a pipeline of NLP functions
import spacy
from spacy.tokens import Span
from spacy import displacy

# Shapely is used to work with geometric shapes
from shapely.geometry import Point

# Fuzzywuzzy is used for fuzzy searches
from fuzzywuzzy import fuzz

# used for the checklist
import ipywidgets as widgets

# Data preparation <a class="anchor" id="section-datapreparation"></a>

You also want to set up some directories for the import and output of data.

In [2]:
## Declare the data directories
## This presumes that Notebooks/ is the current working directory  
text_directory = os.path.normpath("../Texts/")
csv_directory = os.path.normpath("../ner_output/")
reference_directory = os.path.normpath("../Data")
#maps_directory = os.path.normpath("../maps/")

## Create the data directories
if not os.path.exists(text_directory):
    os.makedirs(text_directory)
if not os.path.exists(csv_directory):
   os.makedirs(csv_directory)
if not os.path.exists(reference_directory):
   os.makedirs(reference_directory)

#if not os.path.exists(maps_directory):
#    os.makedirs(maps_directory)

### Identifying States and Capitals <a class="anchor" id="section-statescapitals"></a>

Some placenames, like *High Street* or *Maryborough*, may be very common across the world, or even in Australia. However, certain placenames refer to significant locations, like states, territories, large geographic features or capital cities. As such, if they are mentioned in a text, the placename is more likely to refer to the major location than a town or village in Tasmania.

These significant locations are a finite set. They can be defined in a reference file that can be reused when reviewing the placenames of any text.

A good point for you to start is a file about modern capital cities.

In [3]:
filename="Contemporary_Capital_Cities.csv"
print("Working on | ", filename)

# set the specific path for the 'filename' which is basically working through a list of everything that is in the folder
reference_location = os.path.normpath(os.path.join(reference_directory, filename))
reference_filename = os.path.basename(reference_location)

# Using pandas, read the csv file. This will place it in a dataframe format. 
#locationreferences_df=pd.DataFrame(columns=['Contemporary'])

#contemporary_references_df = pd.read_csv(reference_location, encoding="utf-8", header=0)
#contemporary_references_dict = contemporary_references_df.to_dict()

Working on |  Contemporary_Capital_Cities.csv


Rather than reading this and then processing it, you can process each line as you read it.

In [4]:
# Design the new Dataframe to hold the Long data
locref_df= pd.DataFrame(columns=['LocationName','Category', 'Longitude','Latitude','PartOf'])

In [5]:
import csv

# read the file
with open(reference_location, newline='') as reffile:
    reader = csv.DictReader(reffile)
    # process each row
    for row in reader:
        
        # Get the Long Country data
        if row['CountryName']!=None and len(row['CountryName']) > 0:
            locdata_df={'LocationName': row['CountryName'],
                   'Category': 'country',
                   'Longitude': row['Country_Longitude'],
                   'Latitude': row['Country_Latitude'],
                   'PartOf': row['ContinentName']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
        
        # Get the Long Country data
        if row['Country']!=None and len(row['Country']) > 0:
            locdata_df={'LocationName': row['Country'],
                   'Category': 'country',
                   'Longitude': row['Country_Longitude'],
                   'Latitude': row['Country_Latitude'],
                   'PartOf': row['ContinentName']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)

        # Get the Long Capital data
        if row['CapitalName']!=None:
            locdata_df={'LocationName': row['CapitalName'],
                   'Category': 'capital',
                   'Longitude': row['CapitalLongitude'],
                   'Latitude': row['CapitalLatitude'],
                   'PartOf': row['CountryName']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
 
        # Get the Long Continent data
        if row['ContinentName']!=None:
            locdata_df={'LocationName': row['ContinentName'],
                   'Category': 'continent',
                   'Longitude': None,
                   'Latitude': None,
                   'PartOf': None}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)



{'LocationName': 'Afghanistan', 'Category': 'country', 'Longitude': '69.183333', 'Latitude': '34.51666667', 'PartOf': 'Asia'}
{'LocationName': 'Kabul', 'Category': 'capital', 'Longitude': '69.183333', 'Latitude': '34.51666667', 'PartOf': 'Afghanistan'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Aland Islands', 'Category': 'country', 'Longitude': '19.9', 'Latitude': '60.116667', 'PartOf': 'Europe'}
{'LocationName': 'Mariehamn', 'Category': 'capital', 'Longitude': '19.9', 'Latitude': '60.116667', 'PartOf': 'Aland Islands'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Albania', 'Category': 'country', 'Longitude': '19.816667', 'Latitude': '41.31666667', 'PartOf': 'Europe'}
{'LocationName': 'Tirana', 'Category': 'capital', 'Longitude': '19.816667', 'Latitude': '41.31666667', 'PartOf': 'Albania'}
{'LocationName': 'Europe', 'Category': 'co

{'LocationName': 'Thimphu', 'Category': 'capital', 'Longitude': '89.633333', 'Latitude': '27.46666667', 'PartOf': 'Bhutan'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Bolivia', 'Category': 'country', 'Longitude': '-68.15', 'Latitude': '-16.5', 'PartOf': 'South America'}
{'LocationName': 'La Paz', 'Category': 'capital', 'Longitude': '-68.15', 'Latitude': '-16.5', 'PartOf': 'Bolivia'}
{'LocationName': 'South America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Bosnia and Herzegovina', 'Category': 'country', 'Longitude': '18.416667', 'Latitude': '43.86666667', 'PartOf': 'Europe'}
{'LocationName': 'Sarajevo', 'Category': 'capital', 'Longitude': '18.416667', 'Latitude': '43.86666667', 'PartOf': 'Bosnia and Herzegovina'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Botswana', 'Category'

{'LocationName': 'Havana', 'Category': 'capital', 'Longitude': '-82.35', 'Latitude': '23.11666667', 'PartOf': 'Cuba'}
{'LocationName': 'North America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'CuraÃ§ao', 'Category': 'country', 'Longitude': '-68.916667', 'Latitude': '12.1', 'PartOf': 'North America'}
{'LocationName': 'Willemstad', 'Category': 'capital', 'Longitude': '-68.916667', 'Latitude': '12.1', 'PartOf': 'CuraÃ§ao'}
{'LocationName': 'North America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Cyprus', 'Category': 'country', 'Longitude': '33.366667', 'Latitude': '35.16666667', 'PartOf': 'Europe'}
{'LocationName': 'Nicosia', 'Category': 'capital', 'Longitude': '33.366667', 'Latitude': '35.16666667', 'PartOf': 'Cyprus'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Czech Republic', 'Category': 'country',

{'LocationName': 'Gibraltar', 'Category': 'country', 'Longitude': '-5.35', 'Latitude': '36.13333333', 'PartOf': 'Europe'}
{'LocationName': 'Gibraltar', 'Category': 'capital', 'Longitude': '-5.35', 'Latitude': '36.13333333', 'PartOf': 'Gibraltar'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Greece', 'Category': 'country', 'Longitude': '23.733333', 'Latitude': '37.98333333', 'PartOf': 'Europe'}
{'LocationName': 'Cretan State', 'Category': 'country', 'Longitude': '23.733333', 'Latitude': '37.98333333', 'PartOf': 'Europe'}
{'LocationName': 'Athens', 'Category': 'capital', 'Longitude': '23.733333', 'Latitude': '37.98333333', 'PartOf': 'Greece'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Greenland', 'Category': 'country', 'Longitude': '-51.75', 'Latitude': '64.18333333', 'PartOf': 'Central America'}
{'LocationName': 'Nuuk', 'Category':

{'LocationName': 'Kingston', 'Category': 'capital', 'Longitude': '-76.8', 'Latitude': '18', 'PartOf': 'Jamaica'}
{'LocationName': 'North America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Japan', 'Category': 'country', 'Longitude': '139.75', 'Latitude': '35.68333333', 'PartOf': 'Asia'}
{'LocationName': 'Republic of Ezo', 'Category': 'country', 'Longitude': '139.75', 'Latitude': '35.68333333', 'PartOf': 'Asia'}
{'LocationName': 'Tokyo', 'Category': 'capital', 'Longitude': '139.75', 'Latitude': '35.68333333', 'PartOf': 'Japan'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Jersey', 'Category': 'country', 'Longitude': '-2.1', 'Latitude': '49.18333333', 'PartOf': 'Europe'}
{'LocationName': 'Saint Helier', 'Category': 'capital', 'Longitude': '-2.1', 'Latitude': '49.18333333', 'PartOf': 'Jersey'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': Non

{'LocationName': 'Africa', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Mauritius', 'Category': 'country', 'Longitude': '57.483333', 'Latitude': '-20.15', 'PartOf': 'Africa'}
{'LocationName': 'Port Louis', 'Category': 'capital', 'Longitude': '57.483333', 'Latitude': '-20.15', 'PartOf': 'Mauritius'}
{'LocationName': 'Africa', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Mexico', 'Category': 'country', 'Longitude': '-99.133333', 'Latitude': '19.43333333', 'PartOf': 'Central America'}
{'LocationName': 'Mexico City', 'Category': 'capital', 'Longitude': '-99.133333', 'Latitude': '19.43333333', 'PartOf': 'Mexico'}
{'LocationName': 'Central America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Moldova', 'Category': 'country', 'Longitude': '28.85', 'Latitude': '47', 'PartOf': 'Europe'}
{'LocationName': 'Chisinau', 'Category': 'capital', 'Long

{'LocationName': 'Oman', 'Category': 'country', 'Longitude': '58.583333', 'Latitude': '23.61666667', 'PartOf': 'Asia'}
{'LocationName': 'Muscat', 'Category': 'capital', 'Longitude': '58.583333', 'Latitude': '23.61666667', 'PartOf': 'Oman'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Pakistan', 'Category': 'country', 'Longitude': '73.05', 'Latitude': '33.68333333', 'PartOf': 'Asia'}
{'LocationName': 'Islamabad', 'Category': 'capital', 'Longitude': '73.05', 'Latitude': '33.68333333', 'PartOf': 'Pakistan'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Palau', 'Category': 'country', 'Longitude': '134.633333', 'Latitude': '7.483333333', 'PartOf': 'Australia'}
{'LocationName': 'Melekeok', 'Category': 'capital', 'Longitude': '134.633333', 'Latitude': '7.483333333', 'PartOf': 'Palau'}
{'LocationName': 'Australia', 'Category': 'continent', 'Long

{'LocationName': 'Dakar', 'Category': 'capital', 'Longitude': '-17.633333', 'Latitude': '14.73333333', 'PartOf': 'Senegal'}
{'LocationName': 'Africa', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Serbia', 'Category': 'country', 'Longitude': '20.5', 'Latitude': '44.83333333', 'PartOf': 'Europe'}
{'LocationName': 'Belgrade', 'Category': 'capital', 'Longitude': '20.5', 'Latitude': '44.83333333', 'PartOf': 'Serbia'}
{'LocationName': 'Europe', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Seychelles', 'Category': 'country', 'Longitude': '55.45', 'Latitude': '-4.616666667', 'PartOf': 'Africa'}
{'LocationName': 'Victoria', 'Category': 'capital', 'Longitude': '55.45', 'Latitude': '-4.616666667', 'PartOf': 'Seychelles'}
{'LocationName': 'Africa', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Sierra Leone', 'Category': 'country', 'Longitude': '-13

{'LocationName': 'Tajikistan', 'Category': 'country', 'Longitude': '68.766667', 'Latitude': '38.55', 'PartOf': 'Asia'}
{'LocationName': 'Dushanbe', 'Category': 'capital', 'Longitude': '68.766667', 'Latitude': '38.55', 'PartOf': 'Tajikistan'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Tanzania', 'Category': 'country', 'Longitude': '39.283333', 'Latitude': '-6.8', 'PartOf': 'Africa'}
{'LocationName': 'Zanzibar, Sultanate of', 'Category': 'country', 'Longitude': '39.283333', 'Latitude': '-6.8', 'PartOf': 'Africa'}
{'LocationName': 'Dar es Salaam', 'Category': 'capital', 'Longitude': '39.283333', 'Latitude': '-6.8', 'PartOf': 'Tanzania'}
{'LocationName': 'Africa', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Thailand', 'Category': 'country', 'Longitude': '100.516667', 'Latitude': '13.75', 'PartOf': 'Asia'}
{'LocationName': 'Bangkok', 'Category': 'capital', 'Longi

{'LocationName': 'South America', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Vietnam', 'Category': 'country', 'Longitude': '105.85', 'Latitude': '21.03333333', 'PartOf': 'Asia'}
{'LocationName': '__i Nam (Nguy_n dynasty)', 'Category': 'country', 'Longitude': '105.85', 'Latitude': '21.03333333', 'PartOf': 'Asia'}
{'LocationName': 'Hanoi', 'Category': 'capital', 'Longitude': '105.85', 'Latitude': '21.03333333', 'PartOf': 'Vietnam'}
{'LocationName': 'Asia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Wallis and Futuna', 'Category': 'country', 'Longitude': '-171.933333', 'Latitude': '-13.95', 'PartOf': 'Australia'}
{'LocationName': 'Mata-Utu', 'Category': 'capital', 'Longitude': '-171.933333', 'Latitude': '-13.95', 'PartOf': 'Wallis and Futuna'}
{'LocationName': 'Australia', 'Category': 'continent', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Western Sahara', 

In [6]:
locref_df

,LocationName,Category,Longitude,Latitude,PartOf
0,Afghanistan,country,69.183333,34.51666667,Asia
1,Kabul,capital,69.183333,34.51666667,Afghanistan
2,Asia,continent,None,None,None
3,Aland Islands,country,19.9,60.116667,Europe
4,Mariehamn,capital,19.9,60.116667,Aland Islands
...,...,...,...,...,...
775,Marshall Islands District,capital,1885,Marshall Islands,"Jabor,ÊJaluit Atoll"
776,moved toÊMajuro Atoll,continent,None,None,None
777,Kolonia,country,1986,Federated States of Micronesia,moved toÊPalikir
778,Federated States of Micronesia,capital,1986,Federated States of Micronesia,Kolonia


The next step is to reorganise this information into a format that is easier to work with. Rather than having country names and captial cities in a Wide data format (i.e., different columns), you can reorganise them into a Long format (i.e., the same column but with another column indicating what geopolitical category they relate to).

Of course, if you are researching historical texts, then some of these contemporary locations may have had different names. Old New York was once New Amsterdam (and had the [nickname of Gotham](https://www.nypl.org/blog/2011/01/25/so-why-do-we-call-it-gotham-anyway), amongst others). Istanbul was Constantinople. Some locations had [romanized names](https://en.wikipedia.org/wiki/Chinese_postal_romanization), like Beijing being called Peking. They may be a long time gone but you might want to add them to the list of significant known locations.

Another historical variant is changing which cities are the capitals. These may be due to political decisions, like the movement of the Australian parliament from Melbourne to the new city of Canberra, or they could be a necessity due to the results of war, like Bonn becoming the capital of West Germany after World War II. These older capitals may also have to be accomodated in your reference data.

Because FtToHNL is set in the 19th Century CE, the next step is to add various capital cities from then.

In [7]:
filename="Capital_Cities_of_19th_Century.csv"
print("Working on | ", filename)

# set the specific path for the 'filename' which is basically working through a list of everything that is in the folder
reference_location = os.path.normpath(os.path.join(reference_directory, filename))
reference_filename = os.path.basename(reference_location)

Working on |  Capital_Cities_of_19th_Century.csv


Again, you need to read the reference file and add any helpful data into your reference list.

In [8]:
# read the file
with open(reference_location, newline='') as reffile:
    reader = csv.DictReader(reffile)
    # process each row
    for row in reader:
        
        # Get the Long Capital data
        if row['Old Capital City']!=None and len(row['Old Capital City']) > 0:
            locdata_df={'LocationName': row['Old Capital City'],
                   'Category': 'capital',
                   'Longitude': None,
                   'Latitude': None,
                   'PartOf': row['Today_A_Part_of']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
        

{'LocationName': 'Kelmis', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Belgium'}
{'LocationName': 'Tarnovo', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Bulgaria'}
{'LocationName': 'Plovdiv', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Bulgaria'}
{'LocationName': 'Phnom Penh', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Cambodia'}
{'LocationName': 'Grand-Bassam', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Ivory Coast'}
{'LocationName': 'Ragusa (Dubrovnik)', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Croatia'}
{'LocationName': 'Boma', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Congo, Democratic Republic of the'}
{'LocationName': 'Copenhagen', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Denmark'}
{'LocationName': 'Levuka', 'Category': 'capital', 'Longitude': None, 'Latitud

There are also larger geopolitical regions that may have been associated with placenames and cultures, for instance empires, dynasties and colonies like the British Empire or the Zulu Kingdom. Again, the borders and applicability of these political entities changed over time, so a contemporary reference list may not include them. 

The 19th Century CE was a time of many European Empires so for FtToHNL, you will need to add reference data associated with relevant entities.

In [9]:
filename="Political_Entities_of_19th_Century.csv"
print("Working on | ", filename)

# set the specific path for the 'filename' which is basically working through a list of everything that is in the folder
reference_location = os.path.normpath(os.path.join(reference_directory, filename))
reference_filename = os.path.basename(reference_location)

Working on |  Political_Entities_of_19th_Century.csv


When processing this reference file, you can add the old political entity, its capital (if known), the geographic region (like continent or part thereof) and the modern country it would be considered part of.  

In [10]:
# read the file
with open(reference_location, newline='') as reffile:
    reader = csv.DictReader(reffile)
    # process each row
    for row in reader:
        
        # Get the Long Capital data
        if row['Capital']!=None and len(row['Capital']) > 0:
            locdata_df={'LocationName': row['Capital'],
                   'Category': 'capital',
                   'Longitude': None,
                   'Latitude': None,
                   'PartOf': row['CountryName']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
 
        # Get the Long Political data
        if row['Name']!=None and len(row['Name']) > 0:
            locdata_df={'LocationName': row['Name'],
                   'Category': 'political',
                   'Longitude': row['Country_Longitude'],
                   'Latitude': row['Country_Latitude'],
                   'PartOf': row['CountryName']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)

        # Get the Long Country data (about the modern Country)
        if row['CountryName']!=None and len(row['CountryName']) > 0:
            locdata_df={'LocationName': row['CountryName'],
                   'Category': 'country',
                   'Longitude': row['Country_Longitude'],
                   'Latitude': row['Country_Latitude'],
                   'PartOf': row['Region']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
        if row['Today Part of']!=None and len(row['Today Part of']) > 0:
            locdata_df={'LocationName': row['Today Part of'],
                   'Category': 'country',
                   'Longitude': row['Country_Longitude'],
                   'Latitude': row['Country_Latitude'],
                   'PartOf': row['Region']}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)

        # Get the Long Region/Continent data
        if row['Region']!=None and len(row['Region']) > 0:
            locdata_df={'LocationName': row['Region'],
                   'Category': 'region',
                   'Longitude': None,
                   'Latitude': None,
                   'PartOf': None}
            print(locdata_df)
            locref_df = locref_df.append(locdata_df, ignore_index=True)
               

{'LocationName': 'Colony, Portuguese', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': ''}
{'LocationName': 'Luanda', 'Category': 'political', 'Longitude': '', 'Latitude': '', 'PartOf': ''}
{'LocationName': 'Angola', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'São Salvador', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Angola'}
{'LocationName': 'Kingdom of Kongo', 'Category': 'political', 'Longitude': '13.216667', 'Latitude': '-8.833333333', 'PartOf': 'Angola'}
{'LocationName': 'Angola', 'Category': 'country', 'Longitude': '13.216667', 'Latitude': '-8.833333333', 'PartOf': 'Africa: Central'}
{'LocationName': 'Angola', 'Category': 'country', 'Longitude': '13.216667', 'Latitude': '-8.833333333', 'PartOf': 'Africa: Central'}
{'LocationName': 'Africa: Central', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Not specified', 'Category': 'capital', 'Lo

{'LocationName': 'Zambia', 'Category': 'country', 'Longitude': '28.283333', 'Latitude': '-15.41666667', 'PartOf': 'Africa: Central'}
{'LocationName': 'Zambia', 'Category': 'country', 'Longitude': '28.283333', 'Latitude': '-15.41666667', 'PartOf': 'Africa: Central'}
{'LocationName': 'Africa: Central', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Sultans of the Comoros', 'Category': 'political', 'Longitude': '43.233333', 'Latitude': '-11.7', 'PartOf': 'Comoros'}
{'LocationName': 'Comoros', 'Category': 'country', 'Longitude': '43.233333', 'Latitude': '-11.7', 'PartOf': 'Africa: East'}
{'LocationName': 'Comoros', 'Category': 'country', 'Longitude': '43.233333', 'Latitude': '-11.7', 'PartOf': 'Africa: East'}
{'LocationName': 'Africa: East', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Antananarivo', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Madagascar'}
{'LocationName

{'LocationName': 'Somalia', 'Category': 'country', 'Longitude': '45.333333', 'Latitude': '2.066666667', 'PartOf': 'Africa: East, Horn'}
{'LocationName': 'Somalia', 'Category': 'country', 'Longitude': '45.333333', 'Latitude': '2.066666667', 'PartOf': 'Africa: East, Horn'}
{'LocationName': 'Africa: East, Horn', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Majeerteen Sultanate', 'Category': 'political', 'Longitude': '45.333333', 'Latitude': '2.066666667', 'PartOf': 'Somalia'}
{'LocationName': 'Somalia', 'Category': 'country', 'Longitude': '45.333333', 'Latitude': '2.066666667', 'PartOf': 'Africa: East, Horn'}
{'LocationName': 'Somalia', 'Category': 'country', 'Longitude': '45.333333', 'Latitude': '2.066666667', 'PartOf': 'Africa: East, Horn'}
{'LocationName': 'Africa: East, Horn', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Hargeisa', 'Category': 'capital', 'Longitude': None, 'Latitude': None,

{'LocationName': 'Algiers', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Algeria'}
{'LocationName': 'Ottoman Algeria', 'Category': 'political', 'Longitude': '3.05', 'Latitude': '36.75', 'PartOf': 'Algeria'}
{'LocationName': 'Algeria', 'Category': 'country', 'Longitude': '3.05', 'Latitude': '36.75', 'PartOf': 'Africa: Northwest'}
{'LocationName': 'Algeria', 'Category': 'country', 'Longitude': '3.05', 'Latitude': '36.75', 'PartOf': 'Africa: Northwest'}
{'LocationName': 'Africa: Northwest', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Fes', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Morocco'}
{'LocationName': 'Sultanate of Morocco', 'Category': 'political', 'Longitude': '-6.816667', 'Latitude': '34.01666667', 'PartOf': 'Morocco'}
{'LocationName': 'Morocco', 'Category': 'country', 'Longitude': '-6.816667', 'Latitude': '34.01666667', 'PartOf': 'Africa: Northwest'}
{'LocationName': 'Morocco'

{'LocationName': 'Africa: South', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Barotseland', 'Category': 'political', 'Longitude': '28.283333', 'Latitude': '-15.41666667', 'PartOf': 'Zambia'}
{'LocationName': 'Zambia', 'Category': 'country', 'Longitude': '28.283333', 'Latitude': '-15.41666667', 'PartOf': 'Africa: South'}
{'LocationName': 'Zambia', 'Category': 'country', 'Longitude': '28.283333', 'Latitude': '-15.41666667', 'PartOf': 'Africa: South'}
{'LocationName': 'Africa: South', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Mthwakazi', 'Category': 'political', 'Longitude': '31.033333', 'Latitude': '-17.81666667', 'PartOf': 'Zimbabwe'}
{'LocationName': 'Zimbabwe', 'Category': 'country', 'Longitude': '31.033333', 'Latitude': '-17.81666667', 'PartOf': 'Africa: South'}
{'LocationName': 'Zimbabwe', 'Category': 'country', 'Longitude': '31.033333', 'Latitude': '-17.81666667', 'PartOf': 'Africa: 

...

{'LocationName': 'Switzerland', 'Category': 'country', 'Longitude': '7.466667', 'Latitude': '46.91666667', 'PartOf': 'Europe: Central'}
{'LocationName': 'Europe: Central', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Switzerland', 'Category': 'political', 'Longitude': '7.466667', 'Latitude': '46.91666667', 'PartOf': 'Switzerland'}
{'LocationName': 'Switzerland', 'Category': 'country', 'Longitude': '7.466667', 'Latitude': '46.91666667', 'PartOf': 'Europe: Central'}
{'LocationName': 'Switzerland', 'Category': 'country', 'Longitude': '7.466667', 'Latitude': '46.91666667', 'PartOf': 'Europe: Central'}
{'LocationName': 'Europe: Central', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Principality of Moldavia', 'Category': 'political', 'Longitude': '26.1', 'Latitude': '44.43333333', 'PartOf': 'Romania'}
{'LocationName': 'Romania', 'Category': 'country', 'Longitude': '26.1', 'Latitude': '44.43333333'

{'LocationName': 'Italy', 'Category': 'country', 'Longitude': '12.483333', 'Latitude': '41.9', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Italy', 'Category': 'country', 'Longitude': '12.483333', 'Latitude': '41.9', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Europe: Southcentral', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Kingdom of Italy (Napoleonic)', 'Category': 'political', 'Longitude': '12.483333', 'Latitude': '41.9', 'PartOf': 'Italy'}
{'LocationName': 'Italy', 'Category': 'country', 'Longitude': '12.483333', 'Latitude': '41.9', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Italy', 'Category': 'country', 'Longitude': '12.483333', 'Latitude': '41.9', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Europe: Southcentral', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Kingdom of Italy', 'Category': 'political', 'Longitude': '12.483333', 'Latitude': '41.9', 'Par

{'LocationName': 'Malta', 'Category': 'country', 'Longitude': '14.5', 'Latitude': '35.88333333', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Europe: Southcentral', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Crown Colony of Malta', 'Category': 'political', 'Longitude': '14.5', 'Latitude': '35.88333333', 'PartOf': 'Malta'}
{'LocationName': 'Malta', 'Category': 'country', 'Longitude': '14.5', 'Latitude': '35.88333333', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Malta', 'Category': 'country', 'Longitude': '14.5', 'Latitude': '35.88333333', 'PartOf': 'Europe: Southcentral'}
{'LocationName': 'Europe: Southcentral', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Rome', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': ''}
{'LocationName': 'Papal States', 'Category': 'political', 'Longitude': '', 'Latitude': '', 'PartOf': ''}
{'LocationName': 'Papal', 'Category':

{'LocationName': 'Europe: West', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Monaco', 'Category': 'political', 'Longitude': '7.416667', 'Latitude': '43.73333333', 'PartOf': 'Monaco'}
{'LocationName': 'Monaco', 'Category': 'country', 'Longitude': '7.416667', 'Latitude': '43.73333333', 'PartOf': 'Europe: West'}
{'LocationName': 'Monaco', 'Category': 'country', 'Longitude': '7.416667', 'Latitude': '43.73333333', 'PartOf': 'Europe: West'}
{'LocationName': 'Europe: West', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Brussels', 'Category': 'capital', 'Longitude': None, 'Latitude': None, 'PartOf': 'Belgium'}
{'LocationName': 'Belgium', 'Category': 'political', 'Longitude': '4.333333', 'Latitude': '50.83333333', 'PartOf': 'Belgium'}
{'LocationName': 'Belgium', 'Category': 'country', 'Longitude': '4.333333', 'Latitude': '50.83333333', 'PartOf': 'Europe: West, Low Countries'}
{'LocationName': 'Belgiu

{'LocationName': 'New Zealand', 'Category': 'country', 'Longitude': '174.783333', 'Latitude': '-41.3', 'PartOf': 'Oceania'}
{'LocationName': 'Oceania', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'Niuean monarchy', 'Category': 'political', 'Longitude': '-169.916667', 'Latitude': '-19.01666667', 'PartOf': 'Niue'}
{'LocationName': 'Niue', 'Category': 'country', 'Longitude': '-169.916667', 'Latitude': '-19.01666667', 'PartOf': 'Oceania'}
{'LocationName': 'Niue', 'Category': 'country', 'Longitude': '-169.916667', 'Latitude': '-19.01666667', 'PartOf': 'Oceania'}
{'LocationName': 'Oceania', 'Category': 'region', 'Longitude': None, 'Latitude': None, 'PartOf': None}
{'LocationName': 'British protectorate of Niue', 'Category': 'political', 'Longitude': '-169.916667', 'Latitude': '-19.01666667', 'PartOf': 'Niue'}
{'LocationName': 'Niue', 'Category': 'country', 'Longitude': '-169.916667', 'Latitude': '-19.01666667', 'PartOf': 'Oceania'}
{'LocationNa

You now have a lot of reference data from a various sources. Some of it might have doubled up. Some of the entries might be talking about the same location but may not have all the data. You will need to clean up the reference data, so that there is only a single entry for each location.

In [307]:
# Delete empty entries (there shouldn't be any but we might have missed a check)
print("Start:",len(locref_df))
# [TO BE DONE]

Start: 2701


In [17]:
# For each location, review any entries
for l in locref_df.sort_values(by='LocationName')['LocationName']:
    print(l)





Abu Dhabi
Abuja
Abéché
Accra
Aceh Sultanate
Adamstown
Addis Ababa
Addis Ababa
Aegina
Afghanistan
Afghanistan
Afghanistan
Afghanistan
Afghanistan
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: Central
Africa: East
Africa: East
Africa: East, Horn
Africa: East, Horn
Africa: East, Horn
Africa: East, Horn
Africa: East, Horn
Africa: East, Horn
Africa: East, Horn
Africa: East, 

Kingdom of Imereti
Kingdom of Italy
Kingdom of Italy (Napoleonic)
Kingdom of Janjero
Kingdom of Jimma
Kingdom of Jolof
Kingdom of Kaffa
Kingdom of Kongo
Kingdom of Koya
Kingdom of Limmu-Ennarea
Kingdom of Loango
Kingdom of Lombardy–Venetia
Kingdom of Luba
Kingdom of Lunda
Kingdom of Mangareva
Kingdom of Mysore
Kingdom of Naples
Kingdom of Nepal
Kingdom of Norway (1814)
Kingdom of Nri
Kingdom of Orungu
Kingdom of Portugal
Kingdom of Prussia
Kingdom of Rarotonga
Kingdom of Romania
Kingdom of Rwanda
Kingdom of Sarawak
Kingdom of Sardinia
Kingdom of Saxony
Kingdom of Serbia
Kingdom of Sikkim
Kingdom of Sine
Kingdom of Spain (1810–1873)
Kingdom of Tahiti
Kingdom of Tonga (1900–1970)
Kingdom of Trinacria: Sicily
Kingdom of Welayta
Kingdom of Westphalia
Kingdom of the Netherlands
Kingdom of the Two Sicilies
Kingdoms of Naples and Sicily
Kingston
Kingston
Kingstown
Kinshasa
Kiribati
Kiribati
Kiribati
Kolonia
Konbaung Dynasty
Konbaung dynasty
Kong Empire
Korea
Korea
Korean Empire
Kosovo
Kosovo


Write the dataframe to file.

In [19]:
filename = "reference_location_data.csv"
save_location = os.path.normpath(os.path.join(reference_directory, filename))
save_filename = os.path.basename(save_location)
print("Saving to data to ",save_location)

Saving to data to  ../Data/reference_location_data.csv


In [35]:
locref_df.sort_values(by='LocationName').to_csv(save_location,index=False)